In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time, requests
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [138]:
basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

In [139]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)

In [140]:
akas = pd.read_csv(akas_url,sep='\t', low_memory=False)

In [141]:
ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [142]:
basics = basics.replace({'\\N':np.nan}) 
akas = akas.replace({'\\N':np.nan}) 
ratings = ratings.replace({'\\N':np.nan}) 

In [143]:
basics.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9050185 entries, 0 to 9050184
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 621.4+ MB


In [144]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [145]:
basics['runtimeMinutes'].isna().sum()

6612507

In [146]:
basics= basics.dropna(subset=['runtimeMinutes'])

In [147]:
basics['runtimeMinutes'].isna().sum()

0

In [148]:
basics['genres'].isna().sum()

67030

In [149]:
basics=basics.dropna(subset=['genres'])

In [150]:
basics['genres'].isna().sum()

0

In [151]:
basics['titleType'].value_counts()

tvEpisode       1048278
short            569316
movie            363768
video            173815
tvMovie           87885
tvSeries          85856
tvSpecial         16275
tvMiniSeries      15823
tvShort            9341
videoGame           291
Name: titleType, dtype: int64

In [152]:
movie_filter= basics['titleType']!='movie'

In [153]:
basics= basics.drop(basics[movie_filter].index,inplace=False)

In [154]:
basics['titleType'].value_counts()

movie    363768
Name: titleType, dtype: int64

In [155]:
start_date= '2000'
end_date='2022'

In [156]:
after_start_date = basics['startYear'] >= start_date

In [157]:
before_end_date = basics['startYear'] <= end_date

In [158]:
between_two_dates = after_start_date & before_end_date

In [159]:
filtered_dates = basics.loc[between_two_dates]

In [160]:
filtered_dates

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77968,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
...,...,...,...,...,...,...,...,...,...
9049950,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"
9050034,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
9050075,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
9050102,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary


In [161]:
basics= filtered_dates

In [162]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212780 entries, 13082 to 9050135
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          212780 non-null  object
 1   titleType       212780 non-null  object
 2   primaryTitle    212780 non-null  object
 3   originalTitle   212780 non-null  object
 4   isAdult         212780 non-null  object
 5   startYear       212780 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  212780 non-null  object
 8   genres          212780 non-null  object
dtypes: object(9)
memory usage: 16.2+ MB


In [163]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]


In [164]:
is_documentary.sum()

72120

In [165]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140660 entries, 34805 to 9050034
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          140660 non-null  object
 1   titleType       140660 non-null  object
 2   primaryTitle    140660 non-null  object
 3   originalTitle   140660 non-null  object
 4   isAdult         140660 non-null  object
 5   startYear       140660 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  140660 non-null  object
 8   genres          140660 non-null  object
dtypes: object(9)
memory usage: 10.7+ MB


In [166]:
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34805      True
61119      True
67672      True
77968      True
86806      True
           ... 
9049857    True
9049866    True
9049905    True
9049950    True
9050034    True
Name: tconst, Length: 140660, dtype: bool

In [167]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77968,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86806,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9049857,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9049866,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,90,"Action,Adventure,Thriller"
9049905,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,NaN,84,Thriller
9049950,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"


# API Querry

In [168]:
pip install tmdbsimple

Note: you may need to restart the kernel to use updated packages.


In [169]:
import tmdbsimple as tmdb

In [170]:
with open('/Users/edwardfakier/.secret/imdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [171]:
tmdb.API_KEY =  login['api-key']

In [172]:
def get_movie_with_rating(movie_id):
    
    movie = tmdb.Movies(movie_id)
    releases = movie.releases()
    movie_info= movie.info()
    for c in releases['countries']:
        if c['iso_3166_1']=='US':
            movie_info['certification']= c['certification']
    return movie_info

In [173]:
test = get_movie_with_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 242.789,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [174]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title.akas.tsv.gz',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 '.ipynb_checkpoints']

In [182]:
YEARS_TO_GET = [2000,2001]

In [183]:
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
if file_exists == False:
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [185]:
#Saving new year as the current df
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst'].copy()#.to_list()


In [186]:
previous_df = pd.read_json(JSON_FILE)

In [187]:
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [188]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [189]:
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue

Movies from 2001: 0it [00:00, ?it/s]